In [1]:
import tensorflow as tf
import cosmopower as cp
import yaml
from classy_sz import Class
import numpy as np
import pyDOE as pyDOE
import os

# Preliminary set-up for generating spectra for cosmopower training

## Read the yaml file

In [2]:
path_to_cosmopower_dir = '/Users/boris/Work/CLASS-SZ/SO-SZ/cosmopower'

data_dir_name = 'ACTPol_lite_DR4_baseLCDM_taup_hip'

cobaya_yaml_file = path_to_cosmopower_dir+'/cosmopower/training/spectra_generation_scripts/yaml_files/ACTPol_lite_DR4_baseLCDM_taup_hip.yaml'

try:
    os.mkdir(path_to_cosmopower_dir+'/cosmopower/training/training_data/'+data_dir_name)
except FileExistsError:
    print("File exist")
    
try:
    os.mkdir(path_to_cosmopower_dir+'/cosmopower/training/training_data/'+data_dir_name+'/TT')
except FileExistsError:
    print("File exist")

try:
    os.mkdir(path_to_cosmopower_dir+'/cosmopower/training/training_data/'+data_dir_name+'/TE')
except FileExistsError:
    print("File exist")

try:
    os.mkdir(path_to_cosmopower_dir+'/cosmopower/training/training_data/'+data_dir_name+'/EE')
except FileExistsError:
    print("File exist")
    
try:
    os.mkdir(path_to_cosmopower_dir+'/cosmopower/training/training_data/'+data_dir_name+'/PP')
except FileExistsError:
    print("File exist")

# get path to the folder of this script
# folder_path = os.path.abspath(os.path.dirname(__file__))
folder_path = path_to_cosmopower_dir+'/cosmopower/training/training_data/'+data_dir_name

File exist
File exist
File exist
File exist
File exist


In [3]:
with open(cobaya_yaml_file) as f:
    dict_from_yaml_file = yaml.load(f,Loader=yaml.FullLoader)

In [4]:
cosmo_params = ['logA','n_s','theta_s_1e2','omega_b','omega_cdm','tau_reio']
cosmo_params_names_in_class = ['ln10^{10}A_s','n_s','100*theta_s','omega_b','omega_cdm','tau_reio']

# last column of Table 1 of https://arxiv.org/pdf/1807.06209.pdf
p18_sigmas = [0.014,0.0042,0.00031,0.00015,0.0012,0.0074]
p18_values = [3.043,0.9652,1.04089,0.02233,0.1198,0.0540]

set_width_from_plc18 = True


derived_params_names = ['h',
                        'sigma8',
                        'YHe',
                        'z_reio',
                        'Neff',
                        'tau_rec',
                        'z_rec',
                        'rs_rec',
                        'ra_rec',
                        'tau_star',
                        'z_star',
                        'rs_star',
                        'ra_star']

In [5]:
param_list_all = []
param_list_varied = {}
param_list_varied_cosmo = {}
param_list_other = []
pdict = dict_from_yaml_file['params']
for (k,v) in zip(pdict.keys(),pdict.values()):
    #print(k,v)
    #print(k)
    if 'prior' in v:
        #print('varied param')
#         print(v['prior'])
        param_list_varied[k] = {}
        param_list_varied[k]['bounds'] ={}
        if ('min' and 'max') in v['prior']:
#             print(v['prior']['min'])
            param_list_varied[k]['bounds']={'min':v['prior']['min'],'max':v['prior']['max']}
        elif 'dist' in v['prior']:
#             print(k,v['prior'])
            param_list_varied[k]['bounds']={'min':v['prior']['loc']-5.*v['prior']['scale'],'max':v['prior']['loc']+5.*v['prior']['scale']}
            if k == 'tau_reio':
                if param_list_varied[k]['bounds']['min']<0:
                    param_list_varied[k]['bounds'] = {'min':0.001,'max':v['prior']['loc']+5.*v['prior']['scale']}
                    
    else:
        param_list_other.append(k)

for (p,v) in zip(param_list_varied.keys(),param_list_varied.values()):
    if p in cosmo_params:
#         param_list_varied_cosmo[p]['bounds']={}
#         param_list_varied_cosmo[p]['bounds']=v
        param_list_varied_cosmo[p]=v
        param_list_varied_cosmo[p]['class_name'] = {}
        param_list_varied_cosmo[p]['class_name'] = cosmo_params_names_in_class[cosmo_params.index(p)]


if set_width_from_plc18:
    for p in param_list_varied_cosmo.keys():
        param_list_varied_cosmo[p]['bounds']['min'] = p18_values[cosmo_params.index(p)]-20.*p18_sigmas[cosmo_params.index(p)]
        param_list_varied_cosmo[p]['bounds']['max'] = p18_values[cosmo_params.index(p)]+20.*p18_sigmas[cosmo_params.index(p)]
        if p == 'tau_reio':
            if param_list_varied_cosmo[p]['bounds']['min']<0:
                param_list_varied_cosmo[p]['bounds']['min'] = 0.01


param_list_varied_cosmo
# number of parameters and samples
n_params = len(param_list_varied_cosmo)
n_samples = 12 #int(sys.argv[1])
n_processes  = 4 #int(sys.argv[2])
n_samples_per_process = int(n_samples/n_processes)

## Testing the lh bounds

In [41]:
# Function to generate all binary strings
def generateAllBinaryStrings(n, arr, i,combs):
    if i == n:
        #print(len(arr),arr)
        combs.append(arr.copy())
        return

    arr[i] = 0
    generateAllBinaryStrings(n, arr, i + 1,combs)

    arr[i] = 1
    generateAllBinaryStrings(n, arr, i + 1,combs)
    
n = 6
arr = [None] * n
combs = []

generateAllBinaryStrings(n, arr, 0,combs)

In [236]:
for ic in range(len(combs)):
    class_params_dict = {}
    for ik,k in enumerate(param_list_varied_cosmo.keys()):
        if combs[ic][ik] == 0:
            class_params_dict[param_list_varied_cosmo[k]['class_name']] = param_list_varied_cosmo[k]['bounds']['min']
        else:
            class_params_dict[param_list_varied_cosmo[k]['class_name']] = param_list_varied_cosmo[k]['bounds']['max']
    print(class_params_dict)

    cosmo = Class()
    lmax = dict_from_yaml_file['theory']['classy']['extra_args']['l_max_scalars']
    # Define cosmology (what is not specified will be set to CLASS default parameters)
    params = {'output': 'tCl pCl lCl',
              'lensing': 'yes',
              'l_max_scalars': lmax
              }

    cosmo.set(params)
    cosmo.set(class_params_dict)
    # cosmo.set(classy_precision)
    cosmo.compute()
    print('done')

{'ln10^{10}A_s': 2.763, 'n_s': 0.8812, '100*theta_s': 1.03469, 'omega_b': 0.01933, 'omega_cdm': 0.09580000000000001, 'tau_reio': 0.01}
done
{'ln10^{10}A_s': 2.763, 'n_s': 0.8812, '100*theta_s': 1.03469, 'omega_b': 0.01933, 'omega_cdm': 0.09580000000000001, 'tau_reio': 0.202}
done
{'ln10^{10}A_s': 2.763, 'n_s': 0.8812, '100*theta_s': 1.03469, 'omega_b': 0.01933, 'omega_cdm': 0.1438, 'tau_reio': 0.01}
done
{'ln10^{10}A_s': 2.763, 'n_s': 0.8812, '100*theta_s': 1.03469, 'omega_b': 0.01933, 'omega_cdm': 0.1438, 'tau_reio': 0.202}
done
{'ln10^{10}A_s': 2.763, 'n_s': 0.8812, '100*theta_s': 1.03469, 'omega_b': 0.02533, 'omega_cdm': 0.09580000000000001, 'tau_reio': 0.01}
done
{'ln10^{10}A_s': 2.763, 'n_s': 0.8812, '100*theta_s': 1.03469, 'omega_b': 0.02533, 'omega_cdm': 0.09580000000000001, 'tau_reio': 0.202}
done
{'ln10^{10}A_s': 2.763, 'n_s': 0.8812, '100*theta_s': 1.03469, 'omega_b': 0.02533, 'omega_cdm': 0.1438, 'tau_reio': 0.01}
done
{'ln10^{10}A_s': 2.763, 'n_s': 0.8812, '100*theta_s': 1.

done
{'ln10^{10}A_s': 3.3230000000000004, 'n_s': 1.0492, '100*theta_s': 1.04709, 'omega_b': 0.02533, 'omega_cdm': 0.09580000000000001, 'tau_reio': 0.202}
done
{'ln10^{10}A_s': 3.3230000000000004, 'n_s': 1.0492, '100*theta_s': 1.04709, 'omega_b': 0.02533, 'omega_cdm': 0.1438, 'tau_reio': 0.01}
done
{'ln10^{10}A_s': 3.3230000000000004, 'n_s': 1.0492, '100*theta_s': 1.04709, 'omega_b': 0.02533, 'omega_cdm': 0.1438, 'tau_reio': 0.202}
done


## Set up wanted derived parameters

In [42]:
ic = 0
class_params_dict = {}
for ik,k in enumerate(param_list_varied_cosmo.keys()):
    if combs[ic][ik] == 0:
        class_params_dict[param_list_varied_cosmo[k]['class_name']] = param_list_varied_cosmo[k]['bounds']['min']
    else:
        class_params_dict[param_list_varied_cosmo[k]['class_name']] = param_list_varied_cosmo[k]['bounds']['max']
print(class_params_dict)

cosmo = Class()
lmax = dict_from_yaml_file['theory']['classy']['extra_args']['l_max_scalars']
# Define cosmology (what is not specified will be set to CLASS default parameters)
params = {'output': 'tCl pCl lCl mPk',
          'lensing': 'yes',
          'l_max_scalars': lmax
          }

cosmo.set(params)
cosmo.set(class_params_dict)
# cosmo.set(classy_precision)
cosmo.compute()
print('done')
    

{'ln10^{10}A_s': 2.763, 'n_s': 0.8812, '100*theta_s': 1.03469, 'omega_b': 0.01933, 'omega_cdm': 0.09580000000000001, 'tau_reio': 0.01}
done


In [43]:

derp = {}
for p in derived_params_names:
    derp.update(cosmo.get_current_derived_parameters([p]))


derp


{'h': 0.7280517,
 'sigma8': 0.6225341755994319,
 'YHe': 0.24397050859335379,
 'z_reio': 2.2746963500976562,
 'Neff': 3.0460000000000003,
 'tau_rec': 295.2505191940784,
 'z_rec': 1090.4809846009816,
 'rs_rec': 153.90111589573763,
 'ra_rec': 14874.138913969879,
 'tau_star': 295.1586607345968,
 'z_star': 1090.9241878496293,
 'rs_star': 153.85833768262367,
 'ra_star': 14874.23077262171}

## Generate the latin hyper cube of parameters

In [44]:

print('doing %d times %d calculations'%(n_processes,n_samples_per_process))

for (k,v) in zip(param_list_varied_cosmo.keys(),param_list_varied_cosmo.values()):
    param_list_varied_cosmo[k]['array'] = {}
    param_list_varied_cosmo[k]['array'] = np.linspace(param_list_varied_cosmo[k]['bounds']['min'],param_list_varied_cosmo[k]['bounds']['max'],n_samples)
    

doing 4 times 3 calculations


In [45]:
all_params_list = []
for (k,v) in zip(param_list_varied_cosmo.keys(),param_list_varied_cosmo.values()):
    all_params_list.append(v['array'])

AllParams = np.vstack(all_params_list)
lhd = pyDOE.lhs(n_params, samples=n_samples, criterion=None)
idx = (lhd * n_samples).astype(int)

AllCombinations = np.zeros((n_samples, n_params))
for i in range(n_params):
    AllCombinations[:, i] = AllParams[i][idx[:, i]]
AllCombinations

array([[3.22118182, 0.97283636, 1.04370818, 0.02314818, 0.10452727,
        0.07981818],
       [2.763     , 0.92701818, 1.03919909, 0.02478455, 0.1438    ,
        0.04490909],
       [3.11936364, 0.89647273, 1.03694455, 0.02260273, 0.11325455,
        0.18454545],
       [2.96663636, 0.94229091, 1.04709   , 0.01987545, 0.12198182,
        0.09727273],
       [2.91572727, 0.8812    , 1.04596273, 0.02096636, 0.12634545,
        0.01      ],
       [2.86481818, 0.98810909, 1.04145364, 0.02369364, 0.11761818,
        0.202     ],
       [2.81390909, 0.91174545, 1.04483545, 0.02533   , 0.13943636,
        0.02745455],
       [3.27209091, 1.03392727, 1.03581727, 0.01933   , 0.0958    ,
        0.06236364],
       [3.323     , 0.95756364, 1.03469   , 0.02151182, 0.10889091,
        0.16709091],
       [3.17027273, 1.0492    , 1.04032636, 0.02423909, 0.13070909,
        0.14963636],
       [3.01754545, 1.00338182, 1.03807182, 0.02205727, 0.10016364,
        0.11472727],
       [3.06845455, 1

In [46]:
# saving
lh_param_dict = {}
ip = 0
for (k,v) in zip(param_list_varied_cosmo.keys(),param_list_varied_cosmo.values()):
    lh_param_dict[v['class_name']] = AllCombinations[:,ip]
    ip+=1

    
np.savez(path_to_cosmopower_dir+'/cosmopower/training/training_data/'+data_dir_name+'/LHS_parameter_file.npz', **lh_param_dict)

In [47]:
# saving
for idx_process in range(n_processes):
    params = dict(zip(lh_param_dict.keys(), AllCombinations[idx_process*n_samples_per_process:(idx_process+1)*n_samples_per_process, :].T))
    np.savez(folder_path+'/LHS_parameter_file_{}.npz'.format(idx_process+1), **params)

## Generate spectra

In [6]:
# the running process (index in job array)
# idx_process = sys.argv[1]
idx_process = 1

lmax = dict_from_yaml_file['theory']['classy']['extra_args']['l_max_scalars']
# load parameter file for this process index
params_lhs  = np.load(folder_path+'/LHS_parameter_file_{}.npz'.format(idx_process))



def spectra_generation(idx_sample):
    class_params_dict = {}

    for k in params_lhs:
        class_params_dict[k] = params_lhs[k][idx_sample]
    class_params_dict

    cosmo = Class()

    # Define cosmology (what is not specified will be set to CLASS default parameters)
    params = {'output': 'tCl pCl lCl mPk',
              'lensing': 'yes',
              'l_max_scalars': lmax
              }

    cosmo.set(params)
    cosmo.set(class_params_dict)
    # cosmo.set(classy_precision)
    cosmo.compute()
    cls = cosmo.lensed_cl(lmax=lmax)

    powers  = cls
    clTT    = powers['tt'][2:]
    clTE    = powers['te'][2:]
    clEE    = powers['ee'][2:]
    clPP    = powers['pp'][2:]
    
    derp = {}
    for p in derived_params_names:
        derp.update(cosmo.get_current_derived_parameters([p]))

    cosmo_array_tt = np.hstack(([params_lhs[k][idx_sample] for k in params_lhs],[derp[k] for k in derp.keys()], clTT))
    file_tt           = open(folder_path+'/TT/cls_tt_nointerp_{}.dat'.format(idx_process),'ab')
    np.savetxt(file_tt, [cosmo_array_tt])
    file_tt.close()

    cosmo_array_te = np.hstack(([params_lhs[k][idx_sample] for k in params_lhs],[derp[k] for k in derp.keys()], clTE))
    file_te           = open(folder_path+'/TE/cls_te_nointerp_{}.dat'.format(idx_process),'ab')
    np.savetxt(file_te, [cosmo_array_te])
    file_te.close()

    cosmo_array_ee = np.hstack(([params_lhs[k][idx_sample] for k in params_lhs],[derp[k] for k in derp.keys()], clEE))
    file_ee           = open(folder_path+'/EE/cls_ee_nointerp_{}.dat'.format(idx_process),'ab')
    np.savetxt(file_ee, [cosmo_array_ee])
    file_ee.close()

    cosmo_array_pp = np.hstack(([params_lhs[k][idx_sample] for k in params_lhs],[derp[k] for k in derp.keys()], clPP))
    file_pp           = open(folder_path+'/PP/cls_pp_nointerp_{}.dat'.format(idx_process),'ab')
    np.savetxt(file_pp, [cosmo_array_pp])
    file_pp.close()



# loop over parameter sets in parameter file corresponding to the running process
for i in range(n_samples_per_process):
    spectra_generation(i)

In [252]:
idx_process = 1
# file_pp = open(folder_path+'/PP/cls_pp_nointerp_{}.dat'.format(idx_process),'ab')

In [12]:
f = np.loadtxt(folder_path+'/EE/cls_ee_nointerp_{}.dat'.format(idx_process))

In [14]:
len(f)

3

# Miscellaneous

In [145]:
classy_precision = dict_from_yaml_file['theory']['classy']['extra_args']
classy_precision

{'non linear': 'halofit',
 'N_ncdm': 1,
 'N_ur': 2.0328,
 'P_k_max_h/Mpc': 100.0,
 'z_pk': 1.0,
 'l_max_scalars': 11000,
 'neglect_CMB_sources_below_visibility': 1e-30,
 'transfer_neglect_late_source': 3000.0,
 'halofit_k_per_decade': 3000.0,
 'l_switch_limber': 40.0,
 'accurate_lensing': 1,
 'num_mu_minus_lmax': 1000.0,
 'delta_l_max': 1000.0,
 'k_min_tau0': 0.002,
 'k_max_tau0_over_l_max': 3.0,
 'k_step_sub': 0.015,
 'k_step_super': 0.0001,
 'k_step_super_reduction': 0.1}

In [49]:
cosmo = Class()

# Define cosmology (what is not specified will be set to CLASS default parameters)
params = {'output': 'tCl pCl lCl mPk',
          'lensing': 'yes',
          'l_max_scalars': 11000
          }

cosmo.set(params)
# cosmo.set(classy_precision)
cosmo.compute()
cls = cosmo.lensed_cl(lmax=2508)
spectrum_class = cls['te'][2:]
cosmo.struct_cleanup()
cosmo.empty()

In [100]:
cls


{'tt': array([0.00000000e+00, 0.00000000e+00, 1.56602483e-10, ...,
        9.97822252e-18, 9.94249126e-18, 9.90701731e-18]),
 'ee': array([0.00000000e+00, 0.00000000e+00, 7.65424067e-15, ...,
        3.75760292e-19, 3.75774127e-19, 3.75781028e-19]),
 'te': array([ 0.00000000e+00,  0.00000000e+00,  5.04243172e-13, ...,
        -4.00527435e-19, -4.01059096e-19, -4.01644763e-19]),
 'bb': array([0.00000000e+00, 0.00000000e+00, 2.51434604e-19, ...,
        3.80519722e-21, 3.79759860e-21, 3.79001634e-21]),
 'pp': array([0.00000000e+00, 0.00000000e+00, 9.03616395e-09, ...,
        1.55528788e-22, 1.55147820e-22, 1.54767930e-22]),
 'tp': array([0.00000000e+00, 0.00000000e+00, 5.08391605e-10, ...,
        2.60697120e-23, 2.61610541e-23, 2.62521304e-23]),
 'ell': array([   0,    1,    2, ..., 2506, 2507, 2508])}

In [278]:
dict_from_yaml_file['params'].keys()

dict_keys(['logA', 'A_s', 'n_s', 'theta_s_1e2', '100*theta_s', 'H0', 'omega_b', 'omega_cdm', 'Omega_m', 'omegamh2', 'm_ncdm', 'Omega_Lambda', 'YHe', 'tau_reio', 'yp2', 'z_reio', 'sigma8', 's8h5', 's8omegamp5', 's8omegamp25', 'A', 'clamp', 'age', 'rs_drag'])

In [6]:
# the running process (index in job array)
# idx_process = sys.argv[1]
idx_process = 1

lmax = dict_from_yaml_file['theory']['classy']['extra_args']['l_max_scalars']
# load parameter file for this process index
params_lhs  = np.load(folder_path+'/LHS_parameter_file_{}.npz'.format(idx_process))


idx_sample = 0
# def spectra_generation(idx_sample):
class_params_dict = {}

for k in params_lhs:
    class_params_dict[k] = params_lhs[k][idx_sample]
class_params_dict

cosmo = Class()

# Define cosmology (what is not specified will be set to CLASS default parameters)
params = {'output': 'tCl pCl lCl mPk',
          'lensing': 'yes',
          'l_max_scalars': lmax
          }

cosmo.set(params)
cosmo.set(class_params_dict)
# cosmo.set(classy_precision)
cosmo.compute()
cls = cosmo.lensed_cl(lmax=lmax)

In [9]:
powers  = cls
clTT    = powers['tt'][2:]
clTE    = powers['te'][2:]
clEE    = powers['ee'][2:]
clPP    = powers['pp'][2:]

derp = {}
for p in derived_params_names:
    derp.update(cosmo.get_current_derived_parameters([p]))

cosmo_array_tt = np.hstack(([params_lhs[k][idx_sample] for k in params_lhs],[derp[k] for k in derp.keys()], clTT))
file_tt           = open(folder_path+'/TT/cls_tt_nointerp_{}.dat'.format(idx_process),'ab')
np.savetxt(file_tt, [cosmo_array_tt])
file_tt.close()

In [10]:
f = np.loadtxt(folder_path+'/TT/cls_tt_nointerp_{}.dat'.format(idx_process))

In [13]:
len(f)


11018